In [1]:
# !wget download https://www.dropbox.com/s/tew6i3321ay35ay/GutenTrash.zip?dl=1
# !unzip GutenTrash.zip

In [2]:
import aerospike
config_aerospike = {'hosts': [('35.228.136.58', 3000)]}
client = aerospike.client(config_aerospike).connect()
bert_data_key = ("ids_to_validate", "data", "bert_tags")

In [3]:
from fastai import *
from fastai.vision import *

In [4]:
from tagger import config, train_model, stringConverter
from tagger import predict

/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: thinc.extra.search.Beam size changed, may indicate binary incompatibility. Expected 112 from C header, got 120 from PyObject
  return f(*args, **kwds)


In [5]:
args_ind = {
    "labels": client.get(bert_data_key)[-1]["gutenIndustries"],
    "model": load_learner("models", "guten_industries_P3_v1.pkl"),
}

In [6]:
args_shifts = {
    "labels": client.get(bert_data_key)[-1]["gutenShifts"],
    "model": load_learner("models", "guten_shifts_P3_v1.pkl"),
}

In [11]:
args_trash = {
    "labels": client.get(bert_data_key)[-1]["gutenTrash"],
    "model": load_learner("models", "guten_trash_P3_v1.pkl"),
}

In [7]:
import bson
import pymongo
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags



In [8]:
def set_shifts_industries(_id, industriesPredict, shiftsPredict):
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{
                                             "industries_from_bert": industriesPredict,
                                             "shifts_from_bert": shiftsPredict,
                                    }})    



In [9]:
db = db_client["tweets_pipeline_v2"].find(    
                                        {"$or": [
                                            {"status":"deleted", "source": "dailyshot"},
                                            {"status":"deleted", "source": "twitter", "username": "soberlook"},
                                            {"status":"trasher", "source": "dailyshot"},            
                                            {"status":"trasher", "source": "twitter", "username": "soberlook"}                                                                
                                                                ]})

for i in db:
    _id = str(i["_id"])
    industries_from_bert = predict(stringConverter(i["extracted_text"], use_mister_proper=True), args_ind)
    shifts_from_bert = predict(stringConverter(i["extracted_text"], use_mister_proper=True), args_shifts)
    set_shifts_industries(_id, industries_from_bert, shifts_from_bert)

In [ ]:
db = db_client["tweets_pipeline_v2"].find({"$or":[
                                                      {"status": "validated"},                                                       
                                                 ],
                                                   "weakTag": True,
                                                   "extracted_text": {"$exists": True},                                               
#                                                'tweet_timestamp': {'$gte': datetime(2020,2,4)} #algoStarts                                              
                                          }, no_cursor_timeout=True)

for i in db:
    _id = str(i["_id"])
    industries_from_bert = predict(stringConverter(i["extracted_text"], use_mister_proper=True), args_ind)
    shifts_from_bert = predict(stringConverter(i["extracted_text"], use_mister_proper=True), args_shifts)
    set_shifts_industries(_id, industries_from_bert, shifts_from_bert)

In [23]:
db_client["tweets_pipeline_v2"].find({"status":"daily-shot"}).count()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1000

In [17]:
db = db_client["tweets_pipeline_v2"].find({"$or":[
                                                      {"status": "graphicone_search"},                                                       
                                                 ],
#                                                    "weakTag": True,
                                                   "extracted_text": {"$exists": True},                                               
#                                                'tweet_timestamp': {'$gte': datetime(2020,2,4)} #algoStarts                                              
                                          }, no_cursor_timeout=True)
for i in db:    
    print(i)

{'_id': ObjectId('5dc59bd09f015516f4540941'), 'tweet_text': 'Big Difference In 2 Inflation\xa0Metrics https://t.co/DyUr1c7bOE https://t.co/A8sPRajf98', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2019, 11, 2, 11, 25, 18), 'username': 'vintage_value', 'user_id': 3029451197, 'user_counters': {'followers_count': 10600, 'friends_count': 9757, 'listed_count': 130}, 'tweet_counters': {'retweet_count': 2, 'favorite_count': 1}, 'img_urls': 'http://pbs.twimg.com/media/EIXT75dXUAA-c01.jpg', 'tweet_id': 1190590704143069184, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'US: Personal income and spending Real consumer spending Real disposable income %, year %, year 5 4 4 3 3 2 2 1 1 0 Source: Oxford Economics/Haver Analytics 6', 'graph_type': 'other-chart', 'confirmed_after_validate': ['us', 'personal finance'], 'status': 'graphicone_search', 'validated_timestamp': datetime.datetime(2019, 11, 10, 18, 5, 42, 921000), 'validator_username': 'share10', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
for i in db_client["tweets_pipeline_v2"].find({"status": "deleted",
                                              "extracted_text": {"$exists": True}})[:100]:
    if len(predict(stringConverter(i["extracted_text"], use_mister_proper=False), args)) != 0:
        print(i)
        
#     print(i)
#     print(i["grafeed_shifts"], "|", predict(stringConverter(i["extracted_text"], use_mister_proper=False), args))
#     print()


{'_id': ObjectId('5dc594799f015516f454063d'), 'tweet_text': 'Gold daily chart. \nPrime for a breakout?\n#Gold #Commodities https://t.co/pGdq7pxVnF', 'hashtags': ['Gold', 'Commodities'], 'tweet_timestamp': datetime.datetime(2019, 11, 1, 3, 35), 'username': 'reutersgmf', 'user_id': 1900708207, 'user_counters': {'followers_count': 10805, 'friends_count': 2295, 'listed_count': 552}, 'tweet_counters': {'retweet_count': 1, 'favorite_count': 1}, 'img_urls': 'http://pbs.twimg.com/media/EIQeW76WoAAwbjO.jpg', 'tweet_id': 1190109962543996936, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': '12/25/2017 12/6/2019 (GMT) Daily XAU= Line, XAU-, Bid(Last), 11/1/2019, 1,510.7100, -2.4500, (-0.16 % ) Price USD Ozs F1,530 F1,520 1,510.7100 -1,500 F1,490 - 1,480 1,470 F1,460 F1,450 -1,440 1,430 F1,420 F1,410 -1,400 1,390 1,380 -1,370 F1,360 1,350 1,340 F1,330 1,320 -1,310 1,300 - 1,290 1,280 -1,270 1,260 F1,250 1,240 F1,230 F1,220 1,210 -1,200 1,190 1, 180 Auto RSI, XA

{'_id': ObjectId('5dc59534d7063fa74fe535aa'), 'tweet_text': 'thereby securing 8% returns from now through the year 2525 https://t.co/3q6h8tUxHo', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2019, 10, 28, 19, 20, 55), 'username': 'jasonzweigwsj', 'user_id': 89043072, 'user_counters': {'followers_count': 160222, 'friends_count': 1852, 'listed_count': 2728}, 'tweet_counters': {'retweet_count': 5, 'favorite_count': 23}, 'img_urls': 'http://pbs.twimg.com/media/EH_Q2BrWsAAC-vy.png', 'tweet_id': 1188898457769127938, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': "Chart of the Week: Buyout Valuations Are at a Near 20-Year High A critical determinant of the return on a private equity Purchase Price Multiple in Leveraged Buyouts* Equity Contibution to Leveraged Buyout (% investment is the purchase multiple. That could pose a challenge for current-vintage funds which, in the first half of 2019, paid 11.2 times earnings before interest, taxes, deprec

{'_id': ObjectId('5dc596329f015516f45406fc'), 'tweet_text': 'Wie nu #Proximus verkoopt, ontvangt meer dan #Leroy. https://t.co/FsIYbk1wT6', 'hashtags': ['Proximus', 'Leroy'], 'tweet_timestamp': datetime.datetime(2019, 9, 20, 15, 32, 5), 'username': 'cderijcke', 'user_id': 157284794, 'user_counters': {'followers_count': 3138, 'friends_count': 996, 'listed_count': 90}, 'tweet_counters': {'retweet_count': 3, 'favorite_count': 4}, 'img_urls': 'http://pbs.twimg.com/media/EE6wDe6XsAAiLC3.png', 'tweet_id': 1175070130218393600, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'Proximus €26,62 +0,35 (+1,33%) DELEN GEVOLGD 20-9-2019 1D 5D 1M 1Y 5Y 10Y YTD Bekijk de graflek 26,60 26,50 26,40 26,30 26,20 9:00 10:00 11:00 12:00 13:00 14:00 15:00 16:00 17:00 26,62 26,27 26,22 1,33% 26,64 26,18 НОOG VORIG SLOT HUIDIGE KOERS ОPEN VERSCHIL LAAG', 'graph_type': 'line-area', 'status': 'deleted', 'validated_timestamp': datetime.datetime(2019, 11, 10, 15, 5, 10, 926000)

{'_id': ObjectId('5dc59785d7063fa74fe536bb'), 'tweet_text': 'Check out my latest: “Are U.S. Equities Ready To Breakout?” - https://t.co/NDSM5zMpg9   #IBDpartner\n\n$DJIA $SPX https://t.co/xvi64o7Ys7', 'hashtags': ['IBDpartner'], 'tweet_timestamp': datetime.datetime(2019, 10, 17, 4, 19, 16), 'username': 'andrewnyquist', 'user_id': 302545862, 'user_counters': {'followers_count': 17224, 'friends_count': 264, 'listed_count': 639}, 'tweet_counters': {'retweet_count': 12, 'favorite_count': 23}, 'img_urls': 'http://pbs.twimg.com/media/EHDY_H0WwAAHeZJ.jpg', 'tweet_id': 1184685284874772480, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'Dow Jones Indus Actual P My First List Add to List: Dow Jones Indus Actual (ODJIA) Voume 39216400 6% $26816.59 +242.87 +091% ndes Beae 30000 $DJIA Dow Jones Industrial Average "weekly" chart 278 26951.81 266171 26000 24680.57 23344.52 23000 21169.11 21712.53 20000 18668 44 8351.36 18167.63 17977.85 3000 -Rising wedge patte

{'_id': ObjectId('5dc59883c1c907c0236d0455'), 'tweet_text': '@KyleR_IG Ouch! https://t.co/q3HQxMqYgF', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2019, 10, 24, 20, 23, 47), 'username': 'divyabalji', 'user_id': 37972085, 'user_counters': {'followers_count': 2585, 'friends_count': 1141, 'listed_count': 70}, 'tweet_counters': {'retweet_count': 1, 'favorite_count': 4}, 'img_urls': 'http://pbs.twimg.com/media/EHq44AdWsAACyD2.png', 'tweet_id': 1187464727615672320, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': "AMZN US Equity 1 10/24/2019 96) Actions 10/24/2019 H 16:0020:00 G 480325: Line Chart Overnight Local CCY Range MChart Content Tick K 1D 3D 1M 6M YTD 1Y 5Y Max Shares Slump Amazon tumbles in post-market trading Amazon Close on 10/24 ---- 1780.78 Track Annotate News Zoom 1780 1760 1740 1720 1700 1680 -6.92% '-123.30 1660 17:00 17:30 18:00 19:00 18:30 19:30 20:00 16:30 24 Oct 019", 'graph_type': 'line-area', 'status': 'deleted', 'validated

{'_id': ObjectId('5dc59907c1c907c0236d048d'), 'tweet_text': 'DAX finally completes ascending channel -  https://t.co/04IGwSbfnq https://t.co/CotTIKuv8I', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2017, 6, 20, 20, 44, 54), 'username': 'sttue', 'user_id': 37656736, 'user_counters': {'followers_count': 172, 'friends_count': 330, 'listed_count': 5}, 'tweet_counters': {'retweet_count': 0, 'favorite_count': 0}, 'img_urls': 'http://pbs.twimg.com/media/DCyszWdWAAAHriz.jpg', 'tweet_id': 877266027528286209, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': '13500.0 13000.0 12813.4 12500.0 12000.0 11500.0 11000.0 10500.0 10000.0 9500.0 9000.0 8500.0 Germany 30 8000.0 7500 0 arty', 'graph_type': 'line-area', 'imagehash': 'fefadfc78707ffff', 'confirmed': ['bitcoin', 'cryptocurrency'], 'rejected': [], 'bert_tags': ['bitcoin', 'cryptocurrency'], 'status': 'deleted', 'technical': True, 'dhash': '463636144cca7630', 'phash': 'b5dccd03f3980aaa', 'reason': []

{'_id': ObjectId('5dc59a6bd7063fa74fe53820'), 'tweet_text': 'Silver and Gold Bludgeoned in Controlled\xa0Market https://t.co/6CJ39t924z https://t.co/F9Ln7K71JV', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2018, 6, 26, 16, 15, 16), 'username': 'lawrieongold', 'user_id': 32370221, 'user_counters': {'followers_count': 2589, 'friends_count': 339, 'listed_count': 69}, 'tweet_counters': {'retweet_count': 2, 'favorite_count': 1}, 'img_urls': 'http://pbs.twimg.com/media/DgoU6-xU0AE1PNw.jpg', 'tweet_id': 1011644071419437056, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'S-Saver-Daly Candessck Chat t300 00 J6.800 36 00 e achata 36 300', 'graph_type': 'line-area', 'imagehash': '3fff3f3e8680e0ff', 'confirmed': ['eu', 'us'], 'rejected': [], 'bert_tags': ['eu', 'us'], 'status': 'deleted', 'technical': True, 'dhash': '605ac46434090bb6', 'phash': 'bf87cb7a94432285', 'reason': ['Hourly']}
{'_id': ObjectId('5dc59a79d7063fa74fe53830'), 'tweet_text': 'For

{'_id': ObjectId('5dc59c88d7063fa74fe53920'), 'tweet_text': 'RT @zeroblock: Nothing can stop #Litecoin. WOW. http://t.co/1Lc1xzAouB', 'hashtags': ['Litecoin'], 'tweet_timestamp': datetime.datetime(2013, 11, 26, 19, 47, 13), 'username': 'a_hemphill', 'user_id': 448955285, 'user_counters': {'followers_count': 214, 'friends_count': 868, 'listed_count': 6}, 'tweet_counters': {'retweet_count': 6, 'favorite_count': 0}, 'img_urls': 'http://pbs.twimg.com/media/BaBScVTCQAEPDyD.png', 'tweet_id': 405422510289596417, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'BitcoinWisdom Mt.Gox 896.189 MINING Login or Register Btstamp 57 STC-e 835.e0 BTCChina 865.18/5275.13 LTCUSD 17.9422 MARKETS SETTINGS Time Period 1w 3d 1d 12h 64h 2h 1h 30m 15m Sm 3m 1m РАВАMЕТЕRS AMPLITUDE: 17.74 L 15.28 CHANGE 11.74 V 97169.8 DATE: 2013-11-26 10:et 0: 16.10 H: 17-99 C:17.99 17-99 17.99 17 16 151 14 13 12 11 10 seeeee', 'graph_type': 'line-area', 'imagehash': 'ff0101031e18f010', 'c

{'_id': ObjectId('5dc59d52d7063fa74fe53980'), 'tweet_text': 'Nuevo » Situación del Intermercado. Semana 44-2019 https://t.co/UM6M5PVYiP https://t.co/nDCAby4Zrv', 'hashtags': [], 'tweet_timestamp': datetime.datetime(2019, 11, 6, 11, 21, 32), 'username': 'inbestia', 'user_id': 460138552, 'user_counters': {'followers_count': 15051, 'friends_count': 2996, 'listed_count': 429}, 'tweet_counters': {'retweet_count': 0, 'favorite_count': 1}, 'img_urls': 'http://pbs.twimg.com/media/EIr5bq1UwAADiy4.jpg', 'tweet_id': 1192039306824540160, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'extracted_text': 'CW 44-2019 CAPITAL FLOWS. INTERMARKET Tw @elabarv 201/11/02 chart 01 Bonds Bands EU Indust USA Indust EU Real Estate Cammedites UPrec M Regiens Indust Glebal Size &Style USA Size &Style EU Je Orowh Small Large Farex uSA Gleba N20 US O Techno Techna a ndustrial Real Est Je ndusab HealthC Teco AUtes Real Est China SP 500 tJpan Soxs 600 Growh Comm Sve t.Qualty Jet Lange Low Volat M